In [1]:
import mlb_metrics_helpers

import warnings
warnings.filterwarnings('ignore')

In [2]:
# get batter data
judge_id = mlb_metrics_helpers.player_id("Judge", "Aaron")
judge_general_stats = mlb_metrics_helpers.player_general_metrics(judge_id, timeline_type="career")
judge_timeline = mlb_metrics_helpers.parse_career_timeline(judge_general_stats)

judge_specific_stats = mlb_metrics_helpers.player_specific_metrics(
    judge_id,
    metric_type="batting",
    start_dt=judge_timeline[0],
    end_dt=judge_timeline[1],
)
judge_specific_stats

Gathering player lookup table. This may take a moment.
Gathering Player Data


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,CU,2022-08-12,77.0,-2.27,5.63,"Judge, Aaron",592450,543135,strikeout,swinging_strike,...,1,2,1,2,1,Standard,Standard,41.0,0.023,-0.195
1,CU,2022-08-12,76.8,-2.09,5.59,"Judge, Aaron",592450,543135,NaN,ball,...,1,2,1,2,1,Standard,Standard,47.0,0.000,0.037
2,FC,2022-08-12,89.3,-2.27,5.43,"Judge, Aaron",592450,543135,NaN,ball,...,1,2,1,2,1,Standard,Standard,187.0,0.000,0.017
3,FS,2022-08-12,86.5,-2.38,5.41,"Judge, Aaron",592450,543135,NaN,swinging_strike,...,1,2,1,2,1,Standard,Standard,238.0,0.000,-0.046
4,FF,2022-08-12,92.9,-2.36,5.40,"Judge, Aaron",592450,543135,NaN,called_strike,...,1,2,1,2,1,Standard,Standard,223.0,0.000,-0.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3128,SI,2022-08-13,92.5,-2.87,3.85,"Judge, Aaron",592450,670167,NaN,foul,...,2,2,2,2,2,Standard,Standard,267.0,0.000,-0.042
3129,SL,2022-08-13,86.8,-2.23,5.86,"Judge, Aaron",592450,676710,NaN,ball,...,0,0,0,0,0,Infield shift,Standard,110.0,0.000,0.016
3130,SL,2022-08-13,83.1,-3.15,3.78,"Judge, Aaron",592450,670167,NaN,ball,...,2,2,2,2,2,Standard,Standard,74.0,0.000,0.044
3131,FF,2022-08-13,96.1,-1.98,5.96,"Judge, Aaron",592450,676710,NaN,called_strike,...,0,0,0,0,0,Infield shift,Standard,210.0,0.000,-0.024


In [3]:
model_data = mlb_metrics_helpers.batter_model_metrics(judge_specific_stats)
model_data

,pitch_type,release_speed,release_pos_x,release_pos_y,release_spin_rate,spin_axis,p_throws,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az,description
0,CU,77.0,-2.27,53.95,2070.0,41.0,R,0.82,1.44,5.213895,-111.910639,0.021591,5.075797,23.467290,-40.612124,swinging_strike
3,FS,86.5,-2.38,53.66,1670.0,238.0,R,0.04,1.70,7.816156,-125.822604,-3.344875,-11.776428,21.458979,-28.657600,swinging_strike
5,FF,93.0,-2.42,53.89,2269.0,233.0,R,-0.55,3.76,7.629651,-135.143926,-0.724825,-17.391617,32.104993,-18.524160,hit_into_play
35,SL,87.2,2.31,54.11,2082.0,170.0,L,0.09,2.22,-4.918468,-126.938963,-4.427371,-1.348054,24.203796,-25.082623,hit_into_play
37,FF,94.0,2.33,54.09,2240.0,150.0,L,-0.28,3.72,-8.081169,-136.675580,-4.124875,8.997411,27.747059,-13.121643,swinging_strike
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3102,SL,84.8,-1.81,53.44,2078.0,49.0,R,0.61,1.46,5.207271,-123.467787,-2.670338,1.731825,22.857826,-33.399100,swinging_strike
3107,FF,94.1,-1.29,53.76,2208.0,205.0,R,0.50,1.99,6.105346,-136.665543,-9.777313,-9.394336,30.134973,-10.538146,hit_into_play
3114,FF,95.6,-1.88,54.19,2611.0,212.0,R,-0.21,1.72,6.719506,-138.815237,-9.266855,-14.793779,29.907845,-8.027452,hit_into_play
3117,KC,79.8,-1.84,54.62,2334.0,29.0,R,-0.52,2.02,1.817991,-116.170745,0.326995,4.686705,24.469374,-44.671598,swinging_strike


In [4]:
sklearn_model_types = ["logistic_regression", "random_forest", "gradient_boosting", "hist_gradient_boosting"]
for model_type in sklearn_model_types:
    model, accuracy = mlb_metrics_helpers.tested_batter_model(model_data, model_type)
    print(f"{model_type} accuracy: {accuracy}")

logistic_regression accuracy: 0.6416861826697893
random_forest accuracy: 0.7494145199063232
gradient_boosting accuracy: 0.7564402810304449
hist_gradient_boosting accuracy: 0.7634660421545667
